In [1]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
print('Num devices:', torch.cuda.device_count())

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Num devices: 1


In [3]:
model_id = 'mistralai/Mistral-7B-Instruct-v0.1'

In [3]:
bnb_config = BitsAndBytesConfig(
                # load_in_4bit=True,
                # bnb_4bit_use_double_quant=True,
                # bnb_4bit_quant_type="nf4",
                # bnb_4bit_compute_dtype=torch.bfloat16,
            )
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # quantization_config=bnb_config,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.93s/it]


In [4]:
text = "Hello my name is"
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

outputs = model.generate(input_ids, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!  [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a simple recipe for homemade mayonnaise that you can try:


In [5]:
from datasets import load_dataset
from trl import SFTTrainer
import os
os.environ["WANDB_DISABLED"] = "true"

dataset = load_dataset("imdb", split="train")

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 47.54 GiB of which 87.31 MiB is free. Process 3306651 has 47.45 GiB memory in use. Of the allocated memory 47.04 GiB is allocated by PyTorch, and 104.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [11]:
dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")

Generating train split: 121959 examples [00:00, 125139.14 examples/s]


In [ ]:
def create_text_row(instruction, output, input):
    text_row = f"""<s>[INST] {instruction} here are the inputs {input} [/INST] \\n {output} </s>"""
    return text_row

# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        for item in dataset:
            json_object = {
                "text": create_text_row(item["instruction"], item["input"] ,item["output"]),
                "instruction": item["instruction"],
                "input": item["input"],
                "output": item["output"]
            }
            output_jsonl_file.write(json.dumps(json_object) + "\\n")

# Provide the path where you want to save the formatted dataset
process_jsonl_file("./training_dataset.jsonl")

In [21]:
import os
os.environ["WANDB_DISABLED"] = "true"

# import argparse
import torch
from datasets import Dataset, load_dataset, Features, Value
from transformers import (
    SchedulerType,
    get_scheduler,
    BertTokenizerFast,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding
)
# from peft import AutoPeftModelForCausalLM
# import pandas as pd
from transformers import LlamaForCausalLM, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from transformers.utils import send_example_telemetry
from accelerate.accelerator import Accelerator
from accelerate.tracking import WandBTracker
import accelerate.logging as acc_logging
from accelerate.utils import set_seed, is_wandb_available
from peft import LoraConfig, prepare_model_for_kbit_training
from tqdm.auto import tqdm
# from utils.utils import global_cfg, setup_dirs
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# logger = acc_logging.get_logger(__name__)

# train_defaults = global_cfg['TRAIN_DEFAULTS']
# project_defaults = global_cfg['PROJECT']

# def parse_args():
#     parser = argparse.ArgumentParser(description='Finetune FinBert model')
#     parser.add_argument('--data', dest='data_path', action='store', required=False,
#                         default=train_defaults['data'])
#     parser.add_argument('--output_dir', dest='output_dir', action='store', required=False,
#                         default=train_defaults['output_dir'])
#     parser.add_argument('--model_save_dir', dest='model_save_dir', action='store', required=False,
#                         default=train_defaults['ckpts_dir'])
#     parser.add_argument('--log_dir', dest='log_dir', action='store', required=False, default=train_defaults['log_dir'])
#     parser.add_argument('--log_level', dest='log_level', type=str, default=train_defaults['log_level'])
#     parser.add_argument('--dir_suffix', dest='dir_suffix', type=str, required=False, default='')
#     parser.add_argument('--model_name_or_path', dest='model_name', action='store', required=False,
#                         default=train_defaults['model'])
#     parser.add_argument('--tokenizer_name', dest='tokenizer_name', action='store', required=False,
#                         default=train_defaults['tokenizer'])
#     parser.add_argument('--enable_lora_finetune', dest='enable_lora_finetune', action='store_true', required=False,
#                         default=global_cfg['TRAIN_DEFAULTS.LORA_FINETUNE'].getboolean('enable'))
#     parser.add_argument('--max_length', type=int, default=train_defaults.getint('max_length'))
#     parser.add_argument('--use_slow_tokenizer', action='store_true')
#     parser.add_argument('--per_device_train_batch_size', type=int, default=train_defaults.getint('train_batch_size'))
#     parser.add_argument('--per_device_eval_batch_size', type=int, default=train_defaults.getint('eval_batch_size'))
#     parser.add_argument('--learning_rate', type=float, default=train_defaults.getfloat('lr'))
#     parser.add_argument('--weight_decay', type=float, default=0.0)
#     parser.add_argument('--num_epochs', type=int, default=train_defaults.getint('num_epochs'))
#     parser.add_argument('--gradient_accumulation_steps', type=int, default=1)
#     parser.add_argument('--lr_scheduler', type=SchedulerType, default=train_defaults['lr_scheduler'],
#                         choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant",
#                                  "constant_with_warmup"])
#     parser.add_argument('--num_lr_scheduler_updates_per_epoch', dest='num_lr_scheduler_updates_per_epoch', type=int,
#                         default=train_defaults.getint('num_lr_scheduler_updates_per_epoch'))
#     parser.add_argument('--num_warmup_steps', type=int, default=train_defaults.getint('num_warmup_steps'))
#     parser.add_argument('--no_checkpointing', action='store_true')
#     parser.add_argument('--save_after_num_epochs', dest='save_after_num_epochs', type=int, required=False,
#                         default=train_defaults.getint('save_after_num_epochs'))
#     parser.add_argument('--resume_from_checkpoint', type=str, default=None)
#     parser.add_argument('--reset_optimizer', action='store_true')
#     parser.add_argument('--with_tracking', action='store_true')
#     parser.add_argument('--mixed_precision', action='store_true')
#     parser.add_argument('--seed', type=int, default=None)
#     parser.add_argument('--wandb_proj_name', dest='wandb_proj_name', type=str, default=global_cfg['WANDB']['name'])
#     parser.add_argument('--wandb_run_name', dest='wandb_run_name', type=str, default=None)
#     args = parser.parse_args()

#     return args

# # def compute_metrics(eval_pred):
# #     metric1 = load_metric("precision")
# #     metric2 = load_metric("recall")
    
# #     logits, labels = eval_pred
# #     predictions = np.argmax(logits, axis=-1)
# #     precision = metric1.compute(predictions=predictions, references=labels)["precision"]
# #     recall = metric2.compute(predictions=predictions, references=labels)["recall"]
# #     return {"precision": precision, "recall": recall}


# def train():
#     args = parse_args()
#     config = {
#         'log_dir': args.log_dir,
#         'output_dir': args.output_dir,
#         'ckpts_dir': args.model_save_dir
#     }
#     run_str, log_dir, output_save_path, model_save_path = setup_dirs(config=config,
#                                                                      dir_tag=args.dir_suffix if args.dir_suffix else 'training')

#     if not args.wandb_run_name:
#         args.wandb_run_name = run_str

#     # load datasets
#     # data_schema = Features({
#     # 'Text': Value('string'),    # Assuming 'text' is a key in your JSON containing the text data
#     # })
#     data_files = {'train': '{}/train.jsonl'.format(args.data_path), 'validation': '{}/dev.jsonl'.format(args.data_path)}
    
    
#     # df = pd.read_json('/home/swarna/ET_new_models/dataset/generation/data/train_data_small/train.jsonl', lines=True)
#     # df = df.astype({'Text': str})
#     # train_dataset = Dataset.from_pandas(df)
    
#     datasets = load_dataset('json', data_files=data_files)
#     # train_dataset = Dataset.from_json(data_files['train'], 
#     #                             split='train', cache_dir='{}/datasets'.format(project_defaults['hf_cache_dir']),
#     #                             features=data_schema) 
#     # eval_dataset = Dataset.from_json(data_files['validation'], 
#     #                             split='validation', cache_dir='{}/datasets'.format(project_defaults['hf_cache_dir']),
#     #                             features=data_schema) 
    

datasets = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k")

bnb_configs = BitsAndBytesConfig(
    # load_in_8bit=True
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4'
)


base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_configs,
    trust_remote_code=True,
    device_map="auto"
    # use this if you want to load on accelerate without getting OOM at bitsandbytes function
    # device_map="balanced_low_0"
    
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

base_model = prepare_model_for_kbit_training(base_model)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type='CAUSAL_LM'
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

training_args = TrainingArguments(
    output_dir='output_dir/',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=1,
    fp16=True,
    # ddp_find_unused_parameters=True,
    # fsdp='full_shard',
    # group_by_length=True,
    learning_rate=2e-4,
    logging_steps=10,
    num_train_epochs=10,
    save_strategy='epoch',
    evaluation_strategy="epoch",
    # eval_steps=2,
    log_level="debug",
    optim='paged_adamw_32bit',
    data_seed=768
)


collator = DataCollatorForCompletionOnlyLM(response_template="### Output:", tokenizer=tokenizer)
trainer = SFTTrainer(
    model=base_model,
    train_dataset=datasets['train'],
    # eval_dataset=datasets['test'],
    peft_config=peft_config,
    dataset_text_field="instruction",
    # data_collator=collator,
    # uncomment below if you want to use constantLengthDataset
    packing=False,
    max_seq_length=100,
    tokenizer=tokenizer,
    args=training_args
    # compute_metrics="accuracy"
)

for name, module in trainer.model.named_modules():
    print(f'name: {name}, {type(module)}')
    # if "norm" in name:
    #     module = module.to(torch.float32)

def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))


print_tokens_with_ids(" ### Output:")
print_tokens_with_ids("                ### Output:")
print_tokens_with_ids("### Output:")
print_tokens_with_ids("[Foreign Language] 00:02:37-00:02:46\n                ### Output:\n                Neutral")

print(tokenizer.decode(token_ids=torch.tensor([10961, 8666, 13, 18884,835,10567,29901,13,18884,8680, 5094,   495,  8926,  5381,  7572,   967,   975,  2350,   352,   310,
      967, 16538,   322,  9999,   292, 29892,  3704,   716, 26001, 29892,])))

trainer.train()

trainer.model.save_pretrained('tmp/')


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/73068f3702d050a2fd5aa2ca1e612e5036429398/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.40.0.dev0",
  "use_cache": true,
  "vocab_size": 32000
}

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in 8

name: , <class 'peft.peft_model.PeftModelForCausalLM'>
name: base_model, <class 'peft.tuners.lora.model.LoraModel'>
name: base_model.model, <class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>
name: base_model.model.model, <class 'transformers.models.mistral.modeling_mistral.MistralModel'>
name: base_model.model.model.embed_tokens, <class 'torch.nn.modules.sparse.Embedding'>
name: base_model.model.model.layers, <class 'torch.nn.modules.container.ModuleList'>
name: base_model.model.model.layers.0, <class 'transformers.models.mistral.modeling_mistral.MistralDecoderLayer'>
name: base_model.model.model.layers.0.self_attn, <class 'transformers.models.mistral.modeling_mistral.MistralSdpaAttention'>
name: base_model.model.model.layers.0.self_attn.q_proj, <class 'peft.tuners.lora.bnb.Linear4bit'>
name: base_model.model.model.layers.0.self_attn.q_proj.base_layer, <class 'bitsandbytes.nn.modules.Linear4bit'>
name: base_model.model.model.layers.0.self_attn.q_proj.lora_dropout

***** Running training *****
  Num examples = 20,022
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 50,050
  Number of trainable parameters = 27,262,976
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [14]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [22]:
len([x for x in load_dataset("lucasmccabe-lmi/CodeAlpaca-20k")['train']])

20022